In [4]:
%load_ext autoreload
%autoreload 2
import read_data
import pickle

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
user = "v"
if user == "v":
    sample_data_dir = "/Users/virenbajaj/Desktop/Columbia Fall 20/Graphical Models/project/20200705v1/sample/"
    full_data_dir = "/Users/virenbajaj/Desktop/Columbia Fall 20/Graphical Models/project/HighResLDA/SemanticScholar Data/20200705v1/full/"
else:
    full_data_dir = './Dataset/20200705v1/full/'
    
metadata_dir = full_data_dir + 'metadata/'
pdf_parses_dir = full_data_dir + 'pdf_parses/'

In [6]:
files = range(1)
metadata = [metadata_dir + f'metadata_{i}.jsonl.gz' for i in files]
pdfs = [pdf_parses_dir + f'pdf_parses_{i}.jsonl.gz' for i in files]
# fields = ['Computer Science', 'Medicine']

with open('all_ref_links.pkl', 'rb') as f:
        get_ids = pickle.load(f)
get_ids = set(get_ids) # from cell that gets links of all refs
print(metadata, pdfs)

['/Users/virenbajaj/Desktop/Columbia Fall 20/Graphical Models/project/HighResLDA/SemanticScholar Data/20200705v1/full/metadata/metadata_0.jsonl.gz'] ['/Users/virenbajaj/Desktop/Columbia Fall 20/Graphical Models/project/HighResLDA/SemanticScholar Data/20200705v1/full/pdf_parses/pdf_parses_0.jsonl.gz']


In [8]:
for batch in zip(metadata,pdfs):
    read_data.process_batch(batch[0], batch[1],get_ids=get_ids, output_dir='./processed/refs/')

100%|██████████| 310736/310736 [01:47<00:00, 2884.63it/s]


In [9]:
with open('./processed/refs/text_df/pdf_parses_0.pkl', 'rb') as f:
    text_df = pickle.load(f)
with open('./processed/refs/meta_df/metadata_0.pkl', 'rb') as f:
    meta_df = pickle.load(f)    

In [12]:
text_df

,paper_id,abstract,key_words,body_text,whole_text,citations
0,18980380,This technical note studies Markov decision pr...,"[Distributional robustness, Markov decision p...","[{'section': 'II. PRELIMINARIES', 'text': 'Thr...","Throughout the technical note, we use capital ...",{'BIBREF0': {'title': 'Distributionally robust...
1,16347278,,[],[],,{'BIBREF0': {'title': 'Social and Ethical Inte...
2,54194065,,[],"[{'section': 'D', 'text': '1g1tal Equtpment Co...",1g1tal Equtpment Corporation ts the world's th...,{'BIBREF2': {'title': 'Dvnam•c factor11ahon in...
3,2622106,Abstract-The interference reduction capability...,"[Adaptive beamforming, power control, spaced...","[{'section': 'C', 'text': 'OCHANNEL interferen...",OCHANNEL interference is one of the main impai...,{'BIBREF0': {'title': 'Capacity improvement wi...
4,206643830,Abstract-We analyze and optimize the secrecy p...,"[Artificial noise, multi-input single-output ...","[{'section': 'R', 'text': 'APID and continuous...",APID and continuous growth of wireless mobile ...,{'BIBREF0': {'title': 'Cooperative security at...
5,15326934,,[],"[{'section': 'IV. CONCLUDING REMARKS', 'text':...",The procedure of ship identification from DF s...,{'BIBREF0': {'title': 'An optimal data associa...
6,109247791,,[],[],,{}
7,17668071,. Our framework applied to three different pix...,[],"[{'section': 'Introduction', 'text': 'A number...",A number of computer vision problems can be fo...,{'BIBREF0': {'title': 'Learning to see by movi...
8,11323300,Abstract-Due to the dynamic nature of vehicula...,[],"[{'section': 'I. INTRODUCTION', 'text': 'Vehic...",Vehicular Ad Hoc Networks (VANET) research eff...,{'BIBREF0': {'title': 'Wireless Communications...
9,8808972,,[],[],,{'BIBREF0': {'title': 'ESPRIT-estimation of si...


In [ ]:
# first set text_df to main text_df
text_df_kw = read_data.with_keywords(text_df)

all_citations = (text_df_kw.citations.values)
all_links=[]
all_refs=[]
for citation in all_citations:
    links,refs = read_data.get_links(citation)
    all_links.append(links)
    all_refs.append(refs)

flat_links = [item for sublist in all_links for item in sublist]
links_found = set(flat_links).intersection(set(meta_df.ids)) # links in current df

text_df_kw.set_index('paper_id', inplace=True)

text_df_kw_ref_0 = text_df_kw.loc[list(links_found)]

with open("text_df_kw_ref_0.pkl", 'wb') as f:   
    pickle.dump(text_df_kw_ref_0,f)
    

In [13]:
# Load full dataset
full_data = './Dataset/merged/textdata_all.pkl'
with open(full_data, 'rb') as f:
    full_df = pickle.load(f)

In [20]:
no_key_words = full_df['key_words'][full_df.key_words.str.len() == 0 ]
full_df.loc[no_key_words.index]

,paper_id,abstract,key_words,body_text,whole_text,citations
0,77490118,,[],[],,{}
0,7178153,,[],[],,{'BIBREF0': {'title': 'All-digital phase-domai...
0,2626333,Abstract-Sparse code multiple access (SCMA) is...,"[Non-orthogonal multiple access, SCMA, messa...","[{'section': 'I. INTRODUCTION', 'text': 'T HE ...",T HE GROWING demand for wireless data transmis...,{'BIBREF0': {'title': '5G: A technology vision...
0,17438005,Our objective in this work is to provide a bet...,[],"[{'section': 'Introduction', 'text': 'Computer...",Computer models have been widely used in engin...,"{'BIBREF0': {'title': 'Verification, validatio..."
0,2875476,,[],[],,{'BIBREF0': {'title': 'Band-reweighed Gabor ke...
0,17783159,"This paper presents a new tool, Metro, designe...",[],"[{'section': 'Introduction', 'text': 'Many app...",Many applications produce or manage extremely ...,{'BIBREF0': {'title': 'Geometric computing and...
0,2876646,,[],[],,{'BIBREF0': {'title': 'Clinical application of...
0,16106992,,[],[],,{'BIBREF0': {'title': 'Robust identification o...
0,12940971,,[],[],,{'BIBREF0': {'title': 'A tutorial survey on ve...
0,44337015,,[],[],,{'BIBREF0': {'title': 'A rate control scheme u...


In [18]:
no_key_words = text_df['key_words'][text_df.key_words.str.len() == 0 ]
text_df.loc[no_key_words.index]

,paper_id,abstract,key_words,body_text,whole_text,citations
1,16347278,,[],[],,{'BIBREF0': {'title': 'Social and Ethical Inte...
2,54194065,,[],"[{'section': 'D', 'text': '1g1tal Equtpment Co...",1g1tal Equtpment Corporation ts the world's th...,{'BIBREF2': {'title': 'Dvnam•c factor11ahon in...
5,15326934,,[],"[{'section': 'IV. CONCLUDING REMARKS', 'text':...",The procedure of ship identification from DF s...,{'BIBREF0': {'title': 'An optimal data associa...
6,109247791,,[],[],,{}
7,17668071,. Our framework applied to three different pix...,[],"[{'section': 'Introduction', 'text': 'A number...",A number of computer vision problems can be fo...,{'BIBREF0': {'title': 'Learning to see by movi...
8,11323300,Abstract-Due to the dynamic nature of vehicula...,[],"[{'section': 'I. INTRODUCTION', 'text': 'Vehic...",Vehicular Ad Hoc Networks (VANET) research eff...,{'BIBREF0': {'title': 'Wireless Communications...
9,8808972,,[],[],,{'BIBREF0': {'title': 'ESPRIT-estimation of si...
10,50774180,,[],[],,{'BIBREF0': {'title': 'blankets Eastern Iowa c...
11,407037,In this paper we show that multiple object tra...,[],"[{'section': 'Introduction', 'text': 'Multiple...",Multiple Object Tracking (MOT) is a fundamenta...,{'BIBREF0': {'title': 'TrackingLearning-Detect...
12,22596161,,[],[],,{'BIBREF0': {'title': 'Synchronized Phasor Mea...


In [21]:
full_df.set_index('paper_id', inplace=True)
print(full_df.index.unique())
text_df.set_index('paper_id', inplace=True)
print(text_df.index.unique())
full_corpus = full_df.append(text_df)
print(full_corpus.index.unique())

Index(['18980380', '56031008', '88484504', '88485902', '204197524', '54781908',
       '201906803', '19016491', '54191149', '122282',
       ...
       '51894806', '10266406', '49538416', '16261931', '309940', '4565277',
       '53050558', '1731087', '2119244', '22140957'],
      dtype='object', name='paper_id', length=10090)
Index(['18980380', '16347278', '54194065', '2622106', '206643830', '15326934',
       '109247791', '17668071', '11323300', '8808972',
       ...
       '4957467', '39136868', '16203762', '468828', '39096266', '23466026',
       '30862276', '30865397', '3236236', '16718610'],
      dtype='object', name='paper_id', length=114)
Index(['18980380', '56031008', '88484504', '88485902', '204197524', '54781908',
       '201906803', '19016491', '54191149', '122282',
       ...
       '4957467', '39136868', '16203762', '468828', '39096266', '23466026',
       '30862276', '30865397', '3236236', '16718610'],
      dtype='object', name='paper_id', length=10188)


In [69]:
with open('text_df_kw_ref_0.pkl', 'rb') as f:
    text_df_kw_ref_0 = pickle.load(f)

In [73]:
tdf = text_df_kw_ref_0.dropna()
tdf = tdf.reset_index()

In [82]:
full_data = text_df.append(tdf, sort=True)

In [22]:
with open('final_corpus.pkl', 'wb') as f:
    pickle.dump(full_corpus,f)